# Introduction

## Final Project Submission

***
- Student Name: Adam Marianacci
- Student Pace: Flex
- Scheduled project review date/time: TBD
- Instructor Name: Mark Barbour

# Business Understanding

It is my job to help the WWFA (Water Wells For Africa) locate wells that need to be repaired in Tanzania.

# Data Understanding

# Data Preperation

In [1]:
# Importing the necessary libraries
import pandas as pd
from datetime import datetime
import numpy as np
import seaborn as sns
import folium
import statsmodels as sm
import sklearn
import sklearn.preprocessing as preprocessing
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# Importing the dataframes
df_x = pd.read_csv('data/training_set_values.csv')
df_y = pd.read_csv('data/training_set_labels.csv')

In [4]:
# Combining the 2 dataframes into 1 new dataframe
Waterwells_df = pd.concat([df_y, df_x], axis=1)

In [5]:
# Previewing the dataframe
Waterwells_df.head()

,id,status_group,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [6]:
Waterwells_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   status_group           59400 non-null  object 
 2   id                     59400 non-null  int64  
 3   amount_tsh             59400 non-null  float64
 4   date_recorded          59400 non-null  object 
 5   funder                 55765 non-null  object 
 6   gps_height             59400 non-null  int64  
 7   installer              55745 non-null  object 
 8   longitude              59400 non-null  float64
 9   latitude               59400 non-null  float64
 10  wpt_name               59400 non-null  object 
 11  num_private            59400 non-null  int64  
 12  basin                  59400 non-null  object 
 13  subvillage             59029 non-null  object 
 14  region                 59400 non-null  object 
 15  re

Dropping columns that are not directly related to the business problem and also have high cardinality, making them difficult to one hot encode.

In [7]:
# Dropping irrelevant columns from the dataframe, also columns with large amounts of missing data
columns_to_drop = [
    'id', 'scheme_management', 'region', 'payment', 'public_meeting', 'district_code', 'population', 
    'num_private', 'basin', 'latitude', 'longitude',
    'waterpoint_type_group', 'source_class', 'payment_type', 'management_group', 'recorded_by', 
    'extraction_type', 'management', 
    'source_type', 'extraction_type_group', 'permit', 'funder',
    'date_recorded', 'installer', 'ward', 'scheme_name', 'wpt_name', 'lga', 'subvillage'
]

Waterwells_df = Waterwells_df.drop(columns_to_drop, axis=1, errors='ignore')


In [8]:
# Create a new column 'needs_repair' by merging the two categories
Waterwells_df['needs_repair'] = Waterwells_df['status_group'].replace(
    {'functional': 0, 'non functional': 1, 
     'functional needs repair': 1})

# Drop the original 'status_group' column
Waterwells_df.drop('status_group', axis=1, inplace=True)

#Display the updated DataFrame
Waterwells_df.head()



,amount_tsh,gps_height,region_code,construction_year,extraction_type_class,water_quality,quality_group,quantity,quantity_group,source,waterpoint_type,needs_repair
0,6000.0,1390,11,1999,gravity,soft,good,enough,enough,spring,communal standpipe,0
1,0.0,1399,20,2010,gravity,soft,good,insufficient,insufficient,rainwater harvesting,communal standpipe,0
2,25.0,686,21,2009,gravity,soft,good,enough,enough,dam,communal standpipe multiple,0
3,0.0,263,90,1986,submersible,soft,good,dry,dry,machine dbh,communal standpipe multiple,1
4,0.0,0,18,0,gravity,soft,good,seasonal,seasonal,rainwater harvesting,communal standpipe,0


In [9]:
#dropping the missing values from the 'construction_year' column and creating a new df
Construction_Year_df = Waterwells_df[Waterwells_df['construction_year'] != 0]

# Calculate the current year
current_year = datetime.now().year

# Create a new column 'age' by subtracting construction year from the current year
Construction_Year_df['age'] = current_year - Waterwells_df['construction_year']

In [10]:
Construction_Year_df['construction_year'].value_counts()

2010    2645
2008    2613
2009    2533
2000    2091
2007    1587
2006    1471
2003    1286
2011    1256
2004    1123
2012    1084
2002    1075
1978    1037
1995    1014
2005    1011
1999     979
1998     966
1990     954
1985     945
1996     811
1980     811
1984     779
1982     744
1994     738
1972     708
1974     676
1997     644
1992     640
1993     608
2001     540
1988     521
1983     488
1975     437
1986     434
1976     414
1970     411
1991     324
1989     316
1987     302
1981     238
1977     202
1979     192
1973     184
2013     176
1971     145
1960     102
1967      88
1963      85
1968      77
1969      59
1964      40
1962      30
1961      21
1965      19
1966      17
Name: construction_year, dtype: int64

In [12]:
# deleting the 'construction_year' column since we replaced it with an 'age' column
Construction_Year_df = Construction_Year_df.drop('construction_year', axis=1)

In [13]:
Construction_Year_df.head()

,amount_tsh,gps_height,region_code,extraction_type_class,water_quality,quality_group,quantity,quantity_group,source,waterpoint_type,needs_repair,age
0,6000.0,1390,11,gravity,soft,good,enough,enough,spring,communal standpipe,0,25
1,0.0,1399,20,gravity,soft,good,insufficient,insufficient,rainwater harvesting,communal standpipe,0,14
2,25.0,686,21,gravity,soft,good,enough,enough,dam,communal standpipe multiple,0,15
3,0.0,263,90,submersible,soft,good,dry,dry,machine dbh,communal standpipe multiple,1,38
5,20.0,0,4,submersible,salty,salty,enough,enough,other,communal standpipe multiple,0,15


In [14]:
# Defining X and y variables
y = Construction_Year_df["needs_repair"]
X = Construction_Year_df.drop("needs_repair", axis=1)

In [15]:
# Performing a train, test, split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [16]:
# Looking at the number of missing values in each column
X_train.isna().sum()

amount_tsh               0
gps_height               0
region_code              0
extraction_type_class    0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
waterpoint_type          0
age                      0
dtype: int64

In [17]:
# Create a list of all the categorical features
cols_to_transform = ['quantity_group', 'waterpoint_type','extraction_type_class',
                     'quality_group', 'source',
                     'water_quality', 'quantity']
# Create a dataframe with the new dummy columns created from the cols_to_transform list
X_train = pd.get_dummies(
    data=X_train, columns=cols_to_transform, drop_first=True, dtype=int)

In [18]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30952 entries, 3488 to 24205
Data columns (total 45 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   amount_tsh                                   30952 non-null  float64
 1   gps_height                                   30952 non-null  int64  
 2   region_code                                  30952 non-null  int64  
 3   age                                          30952 non-null  int64  
 4   quantity_group_enough                        30952 non-null  int64  
 5   quantity_group_insufficient                  30952 non-null  int64  
 6   quantity_group_seasonal                      30952 non-null  int64  
 7   quantity_group_unknown                       30952 non-null  int64  
 8   waterpoint_type_communal standpipe           30952 non-null  int64  
 9   waterpoint_type_communal standpipe multiple  30952 non-null  int64  


In [19]:
X_train.head()

,amount_tsh,gps_height,region_code,age,quantity_group_enough,quantity_group_insufficient,quantity_group_seasonal,quantity_group_unknown,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,extraction_type_class_handpump,extraction_type_class_motorpump,extraction_type_class_other,extraction_type_class_rope pump,extraction_type_class_submersible,extraction_type_class_wind-powered,quality_group_fluoride,quality_group_good,quality_group_milky,quality_group_salty,quality_group_unknown,source_hand dtw,source_lake,source_machine dbh,source_other,source_rainwater harvesting,source_river,source_shallow well,source_spring,source_unknown,water_quality_fluoride,water_quality_fluoride abandoned,water_quality_milky,water_quality_salty,water_quality_salty abandoned,water_quality_soft,water_quality_unknown,quantity_enough,quantity_insufficient,quantity_seasonal,quantity_unknown
3488,0.0,1455,16,19,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
12678,200.0,229,5,17,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0
37313,0.0,1588,20,14,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
20930,0.0,1466,13,17,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3639,2000.0,1542,13,34,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0


In [20]:
# Defining the columns to scale
columns_to_scale = ['amount_tsh', 'gps_height']

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the specified columns and transform the data
X_train[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])

In [21]:
# Inspecting the data to make sure it was scaled
X_train.head()

,amount_tsh,gps_height,region_code,age,quantity_group_enough,quantity_group_insufficient,quantity_group_seasonal,quantity_group_unknown,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,extraction_type_class_handpump,extraction_type_class_motorpump,extraction_type_class_other,extraction_type_class_rope pump,extraction_type_class_submersible,extraction_type_class_wind-powered,quality_group_fluoride,quality_group_good,quality_group_milky,quality_group_salty,quality_group_unknown,source_hand dtw,source_lake,source_machine dbh,source_other,source_rainwater harvesting,source_river,source_shallow well,source_spring,source_unknown,water_quality_fluoride,water_quality_fluoride abandoned,water_quality_milky,water_quality_salty,water_quality_salty abandoned,water_quality_soft,water_quality_unknown,quantity_enough,quantity_insufficient,quantity_seasonal,quantity_unknown
3488,0.000000,0.535828,16,19,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
12678,0.000571,0.103071,5,17,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0
37313,0.000000,0.582774,20,14,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
20930,0.000000,0.539711,13,17,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3639,0.005714,0.566537,13,34,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0


# Modeling

In [22]:
# Building a logistic regression model
logreg = LogisticRegression(fit_intercept=False, C=1e12, solver='liblinear')
model_log = logreg.fit(X_train, y_train)
model_log

LogisticRegression(C=1000000000000.0, fit_intercept=False, solver='liblinear')

The classifier was about 72.5% accurate on the training data.

In [23]:
# Checking the performance on the training data
y_hat_train = logreg.predict(X_train)

train_residuals = np.abs(y_train - y_hat_train)
print(pd.Series(train_residuals, name="Residuals (counts)").value_counts())
print()
print(pd.Series(train_residuals, name="Residuals (proportions)").value_counts(normalize=True))

0    23027
1     7925
Name: Residuals (counts), dtype: int64

0    0.743958
1    0.256042
Name: Residuals (proportions), dtype: float64


In [24]:
# Looking at the number of missing values in each column
X_test.isna().sum()

amount_tsh               0
gps_height               0
region_code              0
extraction_type_class    0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
waterpoint_type          0
age                      0
dtype: int64

In [25]:
# Create a list of all the categorical features
cols_to_transform = ['quantity_group', 'waterpoint_type','extraction_type_class',
                     'quality_group', 'source',
                     'water_quality', 'quantity']
# Create a dataframe with the new dummy columns created from the cols_to_transform list
X_test = pd.get_dummies(
    data=X_test, columns=cols_to_transform, drop_first=True, dtype=int)

In [26]:
# Fit the scaler on the specified columns and transform the data
X_test[columns_to_scale] = scaler.fit_transform(X_test[columns_to_scale])

In [27]:
logreg.score(X_test, y_test)

0.7423439720894173

We are still about 74% accuarate on our test data.

In [28]:
y_hat_test = logreg.predict(X_test)

test_residuals = np.abs(y_test - y_hat_test)
print(pd.Series(test_residuals, name="Residuals (counts)").value_counts())
print()
print(pd.Series(test_residuals, name="Residuals (proportions)").value_counts(normalize=True))

0    5745
1    1994
Name: Residuals (counts), dtype: int64

0    0.742344
1    0.257656
Name: Residuals (proportions), dtype: float64


In [29]:
cvscore = cross_val_score(logreg, X_train, y_train.values.ravel(), cv=10)

In [30]:
cvscore

array([0.73837209, 0.750323  , 0.74733441, 0.726979  , 0.74216478,
       0.75218094, 0.74087237, 0.74184168, 0.74927302, 0.74830372])

In [31]:
np.average(cvscore)

0.7437645009663832

In [32]:
np.std(cvscore)

0.007092211801075252

Building a single decision tree

In [33]:
# Create the classifier, fit it on the training data and make predictions on the test set
clf = DecisionTreeClassifier(criterion='entropy')

clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [34]:
y_pred = clf.predict(X_test)

Our True/positives are 3,262 and True/negatives are 2,326. Our False/positives are 1,075 and False/negatives are 1,076

In [36]:
print(confusion_matrix(y_test, y_pred))

[[3262 1075]
 [1076 2326]]


In [37]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75      4337
           1       0.68      0.68      0.68      3402

    accuracy                           0.72      7739
   macro avg       0.72      0.72      0.72      7739
weighted avg       0.72      0.72      0.72      7739



In [38]:
clf.feature_importances_

array([6.03165515e-02, 3.89340000e-01, 8.44749481e-02, 1.53429009e-01,
       8.88103074e-04, 2.10646104e-02, 4.25804328e-02, 3.68805825e-03,
       8.90641504e-03, 1.39802125e-02, 3.12667861e-04, 2.22152726e-03,
       1.51558592e-03, 7.74017178e-02, 6.82054428e-03, 3.95097308e-03,
       7.24513340e-03, 2.78420099e-03, 1.19435165e-02, 3.69857575e-04,
       2.89693953e-04, 4.24835341e-03, 6.47761472e-05, 3.05068360e-03,
       1.57639833e-03, 1.68608953e-03, 1.28308978e-03, 8.24763715e-03,
       2.94629356e-03, 4.75151351e-03, 8.91837885e-03, 7.10367020e-03,
       1.45880987e-02, 5.37483246e-05, 4.15709346e-04, 5.76172000e-05,
       9.50346381e-05, 3.33381375e-03, 8.88501719e-04, 3.65953018e-03,
       4.28793239e-04, 3.81299904e-02, 7.57124643e-04, 1.19946216e-04,
       7.14480068e-05])

In [39]:
print("clf.feature_importances_:", clf.feature_importances_)
print("X.columns:", X_train.columns)

clf.feature_importances_: [6.03165515e-02 3.89340000e-01 8.44749481e-02 1.53429009e-01
 8.88103074e-04 2.10646104e-02 4.25804328e-02 3.68805825e-03
 8.90641504e-03 1.39802125e-02 3.12667861e-04 2.22152726e-03
 1.51558592e-03 7.74017178e-02 6.82054428e-03 3.95097308e-03
 7.24513340e-03 2.78420099e-03 1.19435165e-02 3.69857575e-04
 2.89693953e-04 4.24835341e-03 6.47761472e-05 3.05068360e-03
 1.57639833e-03 1.68608953e-03 1.28308978e-03 8.24763715e-03
 2.94629356e-03 4.75151351e-03 8.91837885e-03 7.10367020e-03
 1.45880987e-02 5.37483246e-05 4.15709346e-04 5.76172000e-05
 9.50346381e-05 3.33381375e-03 8.88501719e-04 3.65953018e-03
 4.28793239e-04 3.81299904e-02 7.57124643e-04 1.19946216e-04
 7.14480068e-05]
X.columns: Index(['amount_tsh', 'gps_height', 'region_code', 'age',
       'quantity_group_enough', 'quantity_group_insufficient',
       'quantity_group_seasonal', 'quantity_group_unknown',
       'waterpoint_type_communal standpipe',
       'waterpoint_type_communal standpipe multipl

In [40]:
features = pd.DataFrame(clf.feature_importances_, index=X_train.columns, columns=['Importance'])
print(features)

                                             Importance
amount_tsh                                     0.060317
gps_height                                     0.389340
region_code                                    0.084475
age                                            0.153429
quantity_group_enough                          0.000888
quantity_group_insufficient                    0.021065
quantity_group_seasonal                        0.042580
quantity_group_unknown                         0.003688
waterpoint_type_communal standpipe             0.008906
waterpoint_type_communal standpipe multiple    0.013980
waterpoint_type_dam                            0.000313
waterpoint_type_hand pump                      0.002222
waterpoint_type_improved spring                0.001516
waterpoint_type_other                          0.077402
extraction_type_class_handpump                 0.006821
extraction_type_class_motorpump                0.003951
extraction_type_class_other                    0

Building a Random Forest Model

In [41]:
rf = RandomForestClassifier()

In [42]:
rf.fit(X_train, y_train)

RandomForestClassifier()

In [43]:
y_pred = rf.predict(X_test)

In [44]:
rf.score(X_test, y_test)

0.7840806305724254

In [45]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.84      0.81      4337
           1       0.78      0.71      0.74      3402

    accuracy                           0.78      7739
   macro avg       0.78      0.78      0.78      7739
weighted avg       0.78      0.78      0.78      7739



In [46]:
features = pd.DataFrame(rf.feature_importances_, index = X_train.columns)
features.head(20)

,0
amount_tsh,0.084192
gps_height,0.341036
region_code,0.088984
age,0.181503
quantity_group_enough,0.023478
quantity_group_insufficient,0.012853
quantity_group_seasonal,0.011523
quantity_group_unknown,0.001590
waterpoint_type_communal standpipe,0.020468
waterpoint_type_communal standpipe multiple,0.011614


In [47]:
#hyper parameters

In [48]:
rf2 = RandomForestClassifier(n_estimators = 1000,
                            criterion = 'entropy',
                            min_samples_split = 10,
                            max_depth = 15,
                            random_state = 42
)

In [49]:
rf2.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=15, min_samples_split=10,
                       n_estimators=1000, random_state=42)

In [50]:
rf2.score(X_test, y_test)

0.7959684713787311

In [51]:
y_pred2 = rf2.predict(X_test)

In [52]:
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.77      0.91      0.83      4337
           1       0.85      0.65      0.74      3402

    accuracy                           0.80      7739
   macro avg       0.81      0.78      0.79      7739
weighted avg       0.80      0.80      0.79      7739



# Evaluation

# Conclusion

# Recommendations

# Limitations

# Next Steps